In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import json
import ast
from sklearn.preprocessing import MinMaxScaler
import matplotlib.cm as cm

# Load the data
df = pd.read_csv('candidates_with_questions.csv')


In [24]:
# Process the answers column to get a numeric matrix
def extract_answers(row):
    # Convert string representation of list to actual list
    answers = ast.literal_eval(row['answers'])
    # Create dictionary with questionId as key and value as value
    return {item['questionId']: item['value']/100 for item in answers}

# Apply extraction to each row
candidates_answers = df.apply(extract_answers, axis=1)

# Create a dataframe with all questions
question_ids = set()
for answers in candidates_answers:
    question_ids.update(answers.keys())
question_ids = sorted(question_ids)


In [25]:
# Create feature matrix
X = np.zeros((len(df), len(question_ids)))
for i, answers in enumerate(candidates_answers):
    for j, q_id in enumerate(question_ids):
        if q_id in answers:
            X[i, j] = answers[q_id]


In [26]:
print(X.shape[1])

58


In [27]:

# Build autoencoder model
input_dim = X.shape[1]
encoding_dim = 4  # Number of dimensions for spider chart

# Encoder
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(32, activation='gelu')(input_layer)
encoded = layers.Dense(16, activation='gelu')(encoded)
encoded = layers.Dense(encoding_dim, activation='sigmoid')(encoded)

# Decoder
decoded = layers.Dense(16, activation='gelu')(encoded)
decoded = layers.Dense(32, activation='gelu')(decoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

# Models
autoencoder = models.Model(input_layer, decoded)
encoder = models.Model(input_layer, encoded)

In [28]:
# Compile and train
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)


autoencoder.compile(optimizer='adam', loss='bce')
autoencoder.fit(X_train, X_train, epochs=100, batch_size=16, shuffle=True, validation_data=(X_test, X_test))

encoded_features = encoder.predict(X_test)

Epoch 1/100
94/94 [==============================] - 1s 2ms/step - loss: 0.6595 - val_loss: 0.6070
Epoch 2/100
94/94 [==============================] - 0s 754us/step - loss: 0.5651 - val_loss: 0.5239
Epoch 3/100
94/94 [==============================] - 0s 791us/step - loss: 0.5076 - val_loss: 0.4848
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4893 - val_loss: 0.4800
Epoch 5/100
94/94 [==============================] - 0s 829us/step - loss: 0.4863 - val_loss: 0.4782
Epoch 6/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4852 - val_loss: 0.4777
Epoch 7/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4844 - val_loss: 0.4771
Epoch 8/100
94/94 [==============================] - 0s 809us/step - loss: 0.4840 - val_loss: 0.4769
Epoch 9/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4835 - val_loss: 0.4773
Epoch 10/100
94/94 [==============================] - 0s 767us/step - loss: 0.4833 - val_loss: 0.4764

In [29]:
last_layer = autoencoder.predict(X_test)

np.testing.assert_almost_equal(X_test/2.5, last_layer/2.5, 1)

12/12 [==============================] - 0s 463us/step


AssertionError: 
Arrays are not almost equal to 1 decimals

Mismatched elements: 2980 / 21808 (13.7%)
Max absolute difference: 0.39426103
Max relative difference: 204.56011051
 x: array([[0. , 0.4, 0. , ..., 0.3, 0.1, 0.3],
       [0. , 0.4, 0. , ..., 0.3, 0.2, 0.4],
       [0.1, 0.4, 0. , ..., 0.3, 0. , 0.3],...
 y: array([[0.1, 0.4, 0. , ..., 0.2, 0.1, 0.4],
       [0.1, 0.4, 0. , ..., 0.3, 0.1, 0.3],
       [0.3, 0.3, 0. , ..., 0.2, 0.1, 0.3],...

In [30]:
display(X)

array([[0.25, 0.75, 0.25, ..., 1.  , 0.33, 0.67],
       [1.  , 0.25, 0.25, ..., 0.67, 0.67, 0.67],
       [0.  , 1.  , 0.  , ..., 0.83, 0.  , 1.  ],
       ...,
       [0.  , 1.  , 0.  , ..., 1.  , 0.5 , 1.  ],
       [0.  , 1.  , 0.  , ..., 0.33, 0.  , 1.  ],
       [0.  , 1.  , 0.  , ..., 0.33, 0.  , 1.  ]])

In [31]:
display(last_layer)

array([[0.16078942, 0.96703744, 0.01406005, ..., 0.58186567, 0.15152209,
        0.9196484 ],
       [0.23045485, 0.93764144, 0.09461331, ..., 0.65611684, 0.25577453,
        0.8667495 ],
       [0.700374  , 0.7542522 , 0.08022233, ..., 0.60707974, 0.20183416,
        0.8369381 ],
       ...,
       [0.4690293 , 0.8395848 , 0.10780191, ..., 0.49986315, 0.24799702,
        0.82928294],
       [0.31832418, 0.8292412 , 0.5843738 , ..., 0.53144795, 0.73232144,
        0.3184727 ],
       [0.9690981 , 0.01807717, 0.7503264 , ..., 0.9673677 , 0.34420663,
        0.4342777 ]], dtype=float32)

In [ ]:

# Create spider charts
def create_spider_chart(features, candidate_name, party):
    # Number of variables
    N = features.shape[0]
    
    # What will be the angle of each axis in the plot
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Close the loop
    
    # Spider chart setup
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    
    # Add the feature values
    values = features.tolist()
    values += values[:1]  # Close the loop
    
    # Plot
    ax.plot(angles, values, linewidth=2, linestyle='solid', label=candidate_name)
    ax.fill(angles, values, alpha=0.25)
    
    # Set labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels([f'Dimension {i+1}' for i in range(N)])
    
    plt.title(f"{candidate_name} ({party}) Political Profile", size=20)
    plt.tight_layout()
    return fig


In [ ]:
for i in range(min(5, len(df))):
    features = encoded_features[i]
    features = (features - features.min()) / (features.max() - features.min() + 1e-10)
    fig = create_spider_chart(features, df.iloc[i]['firstname'] + ' ' + df.iloc[i]['lastname'], 
                              df.iloc[i]['partyAbbreviation'])
    plt.savefig(f"candidate_{i}_spider.png")
    plt.close()


In [ ]:
correlations = []
for dim in range(encoding_dim):
    dim_corr = {}
    for j, q_id in enumerate(question_ids):
        corr = np.corrcoef(encoded_features[:, dim], X[:, j])[0, 1]
        dim_corr[q_id] = corr
    correlations.append(dim_corr)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 376 and the array at index 1 has size 1877

In [ ]:
for dim, corr_dict in enumerate(correlations):
    print(f"Dimension {dim+1} most correlated with:")
    top_corr = sorted(corr_dict.items(), key=lambda x: abs(x[1]), reverse=True)[:5]
    for q_id, corr in top_corr:
        print(f"  Question {q_id}: {corr:.3f}")